In [11]:
# ============================================================
# SUPER SCRIPT (robust HR detection)
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

import os
from pathlib import Path
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# 1. EDIT THESE PATHS
# ------------------------------------------------------------
BASE_DIR      = "/content/drive/MyDrive/Harvard/LS100"
HR_FILE       = f"{BASE_DIR}/data/LS100_Data.xlsx"
ANGLES_FILE   = f"{BASE_DIR}/data/all_angles_master.csv"
OUTPUT_DIR    = f"{BASE_DIR}/analysis"
os.makedirs(OUTPUT_DIR, exist_ok=True)

FPS = 60   # adjust if needed


# ------------------------------------------------------------
# Robust helper: find the HR column
# ------------------------------------------------------------
def find_hr_column(df):
    """Finds the heart-rate column even if name is messy."""
    candidates = [c for c in df.columns if "heart" in c.lower() or "hr" in c.lower()]
    if not candidates:
        raise ValueError(f"No HR-like column found in: {df.columns.tolist()}")
    return candidates[0]  # best guess


# ------------------------------------------------------------
# Robust time converter
# ------------------------------------------------------------
def time_to_seconds(x):
    if isinstance(x, str):
        try:
            return pd.to_timedelta(x).total_seconds()
        except:
            pass
    if hasattr(x, "hour"):  # datetime.time object
        return x.hour * 3600 + x.minute * 60 + x.second
    return np.nan


# ------------------------------------------------------------
# 2. LOAD HR DATA (robust, using the correct 'Time' column)
# ------------------------------------------------------------

HR_SHEETS = {
    "Danny": "Trial 1- Danny",
    "Aryeh": "Trial 2 - Aryeh",
    "Max":   "Trial 3 - Max"
}

hr_data = {}

for person, sheet in HR_SHEETS.items():
    df = pd.read_excel(HR_FILE, sheet_name=sheet)
    df.columns = [c.strip() for c in df.columns]

    # ---- Find HR column ----
    hr_col = None
    for c in df.columns:
        if "heart" in c.lower():
            hr_col = c
            break
    if hr_col is None:
        raise ValueError(f"No HR column found for {person}")

    # ---- Use EXACTLY the 'Time' column ----
    if "Time" not in df.columns:
        raise ValueError(f"'Time' column not found in sheet {sheet}")
    time_col = "Time"

    clean = df[[time_col, hr_col]].copy()
    clean = clean.dropna(subset=[hr_col], how="all")

    # Convert Time → seconds (robust)
    clean["time_s"] = clean[time_col].apply(time_to_seconds)

    # Drop invalid timestamps
    clean = clean.dropna(subset=["time_s"])

    # Standard cleanup
    clean = clean.rename(columns={hr_col: "Heart Rate"})
    clean = clean.sort_values("time_s").reset_index(drop=True)

    hr_data[person] = clean

    print(f"{person} HR loaded: {clean.shape}, columns={clean.columns.tolist()}")

# ------------------------------------------------------------
# 3. LOAD ANGLES FILE
# ------------------------------------------------------------
if Path(ANGLES_FILE).suffix.lower() == ".csv":
    angles = pd.read_csv(ANGLES_FILE)
else:
    angles = pd.read_excel(ANGLES_FILE)

angles.columns = [c.strip() for c in angles.columns]
print("\nLoaded angles with shape:", angles.shape)

required_cols = ["video","frame",
    "angle_left_knee_angle","angle_right_knee_angle",
    "angle_left_hip_angle","angle_right_hip_angle"
]
for c in required_cols:
    if c not in angles.columns:
        raise ValueError(f"Missing {c} in angles file!")


# ------------------------------------------------------------
# 4. Parse video name → participant, trial, clip, phase
# ------------------------------------------------------------
def parse_video_name(v):
    stem = str(Path(v).stem)
    parts = stem.split("_")

    participant = parts[0]
    trial = None
    clip = None

    for p in parts[1:]:
        if p.isdigit() and trial is None:
            trial = p
        if p.startswith("clip"):
            try:
                clip = int(p.replace("clip",""))
            except:
                clip = None

    phase_map = {
        1:"warmup", 2:"mile1", 3:"mile2", 4:"mile3", 5:"cooldown"
    }
    phase = phase_map.get(clip, "unknown")
    return participant, trial, clip, phase

angles[["participant","trial","clip","phase"]] = pd.DataFrame(
    angles["video"].apply(parse_video_name).tolist(),
    index=angles.index
)

# Add angle timestamps
angles["time_s"] = angles["frame"] / FPS


# ------------------------------------------------------------
# 5. MERGE HR + ANGLES for each participant
# ------------------------------------------------------------
merged_frames = []

for person in angles["participant"].unique():
    if person not in hr_data:
        print("Skipping (no HR):", person)
        continue

    A = angles[angles["participant"] == person].sort_values("time_s").copy()
    H = hr_data[person].sort_values("time_s").copy()

    # 🔥 Force both merge keys to the SAME type
    A["time_s"] = A["time_s"].astype(float)
    H["time_s"] = H["time_s"].astype(float)

    merged = pd.merge_asof(
        A, H,
        on="time_s",
        direction="nearest",
        tolerance=1.0
    )

    merged_frames.append(merged)
    print(f"Merged {person}: {merged.shape}")

merged_all = pd.concat(merged_frames, ignore_index=True)
merged_path = f"{OUTPUT_DIR}/merged_hr_angles_all_rows.csv"
merged_all.to_csv(merged_path, index=False)

print("\nSaved full merged dataset →", merged_path)


# ------------------------------------------------------------
# 6. CLIP SUMMARY
# ------------------------------------------------------------
group_cols = ["participant","trial","clip","phase","video"]
angle_cols = [
    "angle_left_knee_angle","angle_right_knee_angle",
    "angle_left_hip_angle","angle_right_hip_angle"
]

agg_dict = {c:["mean","std"] for c in angle_cols}
agg_dict["Heart Rate"] = ["mean","std"]

clip_summary = (
    merged_all.groupby(group_cols)[list(agg_dict.keys())]
    .agg(agg_dict)
    .reset_index()
)

clip_summary.columns = [
    "_".join(col).strip("_") if isinstance(col,tuple) else col
    for col in clip_summary.columns
]

summary_path = f"{OUTPUT_DIR}/clip_summary_with_hr.csv"
clip_summary.to_csv(summary_path, index=False)

print("\nSaved clip summary →", summary_path)
print("\nPreview:")
print(clip_summary.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Danny HR loaded: (2280, 3), columns=['Time', 'Heart Rate', 'time_s']
Aryeh HR loaded: (2284, 3), columns=['Time', 'Heart Rate', 'time_s']
Max HR loaded: (2286, 3), columns=['Time', 'Heart Rate', 'time_s']

Loaded angles with shape: (26880, 6)
Merged Aryeh: (8995, 13)
Merged Danny: (8890, 13)
Merged Max: (8995, 13)

Saved full merged dataset → /content/drive/MyDrive/Harvard/LS100/analysis/merged_hr_angles_all_rows.csv

Saved clip summary → /content/drive/MyDrive/Harvard/LS100/analysis/clip_summary_with_hr.csv

Preview:
  participant trial  clip     phase  \
0       Aryeh     1     1    warmup   
1       Aryeh     1     2     mile1   
2       Aryeh     1     3     mile2   
3       Aryeh     2     4     mile3   
4       Aryeh     2     5  cooldown   

                                           video  angle_left_knee_angle_mean  \
0  Aryeh_1_clip1_pose2d_angles_i